<a href="https://colab.research.google.com/github/AdarshSRM/FirstSite/blob/main/Wally_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import ipywidgets as widgets
from IPython.display import display

# Create the text box and button
url_input = widgets.Text(placeholder='Paste URL here', description='URL:')
button = widgets.Button(description="Set URL", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    global TARGET_URL
    TARGET_URL = url_input.value.strip()
    with output:
        output.clear_output()
        if TARGET_URL:
            print(f"✅ Set to: {TARGET_URL}")
        else:
            print("❌ Please paste a URL first.")

button.on_click(on_button_clicked)

# Display everything
display(url_input, button, output)

Text(value='', description='URL:', placeholder='Paste URL here')

Button(button_style='success', description='Set URL', style=ButtonStyle())

Output()

In [31]:
import shutil
import os

to_remove = ['downloads', 'imagebam_links.txt', 'result_images.zip']

for item in to_remove:
    if os.path.exists(item):
        if os.path.isdir(item):
            shutil.rmtree(item)
            print(f"Removed folder: {item}")
        else:
            os.remove(item)
            print(f"Removed file: {item}")

print("Cleanup complete. Ready for new extraction.")

Removed folder: downloads
Removed file: imagebam_links.txt
Removed file: result_images.zip
Cleanup complete. Ready for new extraction.


In [32]:
import requests
from bs4 import BeautifulSoup

def scrape_links():
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    print(f"Fetching links from: {TARGET_URL}")

    try:
        response = requests.get(TARGET_URL, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract links in order and remove duplicates
        links = [a['href'] for a in soup.find_all('a', href=True) if "imagebam.com/image/" in a['href']]
        unique_links = list(dict.fromkeys(links))

        if unique_links:
            with open("imagebam_links.txt", "w") as f:
                for link in unique_links:
                    f.write(link + "\n")
            print(f"Success! Found {len(unique_links)} links. Saved to 'imagebam_links.txt'.")
        else:
            print("No ImageBam links found.")
    except Exception as e:
        print(f"Error: {e}")

scrape_links()

Fetching links from: https://wallyvideos.blogspot.com/2021/01/debora-natalin-pistarchi-en-el-casting.html
Success! Found 12 links. Saved to 'imagebam_links.txt'.


In [34]:
import requests
from bs4 import BeautifulSoup
import os
import time

os.makedirs("downloads", exist_ok=True)

def resolve_direct_link(session, url):
    # Force HTTPS
    url = url.replace("http://", "https://")

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Referer": "https://www.imagebam.com/"
    }

    try:
        # 1. SET THE MAGIC COOKIES
        # The logs show 'sfw_inter' is the key to bypassing the 'Continue' loop
        session.cookies.set("sfw_inter", "1", domain=".imagebam.com")
        session.cookies.set("nsfw_inter", "1", domain=".imagebam.com")

        # 2. GET the page
        # With the cookies set, this should bypass the interstitial immediately
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 3. SEARCH for the image
        # Using the standard ID and the class found in Imagebam's app.js logic
        img = soup.find('img', id='main-image') or soup.select_one('img.main-image')

        if not img:
            # Fallback: search for any image hosted on their image servers
            img = soup.select_one('img[src*="images2.imagebam.com"]')

        if img:
            return img.get('src')

        return None

    except Exception as e:
        print(f"   [Error]: {e}")
        return None

def download_images():
    if not os.path.exists("imagebam_links.txt"):
        print("Error: imagebam_links.txt not found!")
        return

    with open("imagebam_links.txt", "r") as f:
        urls = [line.strip() for line in f.readlines() if line.strip()]

    session = requests.Session()

    print(f"Downloading {len(urls)} images...")
    for index, url in enumerate(urls, start=1):
        try:
            print(f"[{index}/{len(urls)}] Processing: {url}")
            direct_url = resolve_direct_link(session, url)

            if direct_url:
                # Use the original URL as Referer to avoid 403 Forbidden on the image file
                img_response = session.get(direct_url, headers={"Referer": url})

                original_id = url.split('/')[-1]
                filename = f"downloads/{index}-{original_id}.jpg"

                with open(filename, 'wb') as f:
                    f.write(img_response.content)
                print(f"   -> Saved!")
            else:
                print(f"   -> Failed: Still seeing the 'Continue' page loop.")

            time.sleep(1.2)
        except Exception as e:
            print(f"   -> Error: {e}")

if __name__ == "__main__":
    download_images()

[1/12] Processing: http://www.imagebam.com/image/1630361368539088
   -> Saved!
[2/12] Processing: http://www.imagebam.com/image/e8e6e01368539096
   -> Saved!
[3/12] Processing: http://www.imagebam.com/image/2d27771368539104
   -> Saved!
[4/12] Processing: http://www.imagebam.com/image/a7f0d51368539115
   -> Saved!
[5/12] Processing: http://www.imagebam.com/image/4270d01368539119
   -> Saved!
[6/12] Processing: http://www.imagebam.com/image/09f1ce1368539126
   -> Saved!
[7/12] Processing: http://www.imagebam.com/image/bcc58c1368539136
   -> Saved!
[8/12] Processing: http://www.imagebam.com/image/7d680c1368539141
   -> Saved!
[9/12] Processing: http://www.imagebam.com/image/1ec74b1368539145
   -> Saved!
[10/12] Processing: http://www.imagebam.com/image/411f221368539151
   -> Saved!
[11/12] Processing: http://www.imagebam.com/image/174e151368539157
   -> Saved!
[12/12] Processing: http://www.imagebam.com/image/e29c3f1368539176
   -> Saved!


In [35]:
import os
from google.colab import files

if os.path.exists("downloads") and len(os.listdir("downloads")) > 0:
    print("Zipping files...")
    !zip -q -r result_images.zip downloads
    print("Download starting...")
    files.download('result_images.zip')
else:
    print("Nothing to zip. Make sure the download cell (Cell 4) finished successfully.")

Zipping files...
Download starting...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>